In [1]:
!pip install keras-rl2
!pip install 'gym[all]'

  Using cached https://files.pythonhosted.org/packages/cf/8c/64e0630b3d450244feef0688d90eab2448631e40ba6bdbd90a70b84898e7/mujoco-py-1.50.1.68.tar.gz
  ERROR: Failed building wheel for mujoco-py
  Running setup.py clean for mujoco-py
Failed to build mujoco-py
    Running setup.py install for mujoco-py ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-9m__mn8c/mujoco-py/setup.py'"'"'; __file__='"'"'/tmp/pip-install-9m__mn8c/mujoco-py/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-3e2qcjtl/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [2]:
%%bash

# install required system dependencies
apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists...
Building dependency tree...
Reading state information...
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


In [3]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

In [4]:
import numpy as np

import gym
from gym.envs.registration import registry, register, make, spec

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, Reshape
from tensorflow.keras.optimizers import Adam


from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

from car_racing_v1 import *

In [5]:
register(
    id='CarRacing-v1',
    entry_point='gym.envs.box2d:CarRacing',
    max_episode_steps=2000,
    reward_threshold=900,
)

In [6]:
# Get the environment and extract the number of actions.
ENV_NAME = 'CarRacing-v1'
env = CarRacing()

nb_actions = len(env.action_space)
input_shape = env.observation_space.shape

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
print("nb actions = ", len(env.action_space))
print("observation_space.shape = ", input_shape)

nb actions =  16
observation_space.shape =  (96, 96, 3)


In [8]:
def build_model(input_shape, nb_actions):
    model = Sequential()
    print(input_shape)
    
    model.add(Reshape(input_shape, input_shape = (1,96,96,3)))
    model.add(Convolution2D(32, (8, 8)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (4, 4)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    
    
    '''model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))'''
    
    print(model.summary())
    return model

In [9]:
model = build_model(input_shape, nb_actions)

(96, 96, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 96, 96, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 89, 89, 32)        6176      
_________________________________________________________________
activation (Activation)      (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 86, 86, 64)        32832     
_________________________________________________________________
activation_1 (Activation)    (None, 86, 86, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 84, 84, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 84, 84, 

In [10]:
def build_agent(model, nb_actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=10000)
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=nb_actions, nb_steps_warmup=500, target_model_update=100)
    return dqn

In [11]:
dqn = build_agent(model, nb_actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [12]:
weights_filename = 'dqn_' + ENV_NAME + '_weights.h5f'
checkpoint_weights_filename = 'dqn_' + ENV_NAME + '_weights_{step}.h5f'
log_filename = 'dqn_' + ENV_NAME + '_log.json'
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=2500)]
callbacks += [FileLogger(log_filename, interval=100)]

In [13]:
dqn.fit(env, nb_steps=3000, log_interval=100, verbose=1, nb_max_episode_steps=100, action_repetition=3, visualize = False)

Training for 3000 steps ...
Track generation: 1183..1483 -> 300-tiles track
Interval 1 (0 steps performed)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  3/100 [..............................] - ETA: 3s - reward: -29.4017 Track generation: 1084..1363 -> 279-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1193..1495 -> 302-tiles track
 43/100 [===========>..................] - ETA: 1s - reward: -45.6885Track generation: 1015..1273 -> 258-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1068..1346 -> 278-tiles track
 68/100 [===================>..........] - ETA: 1s - reward: -46.8911Track generation: 1161..1459 -> 298-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1149..1440 -> 291-tiles track
 70/100 [====================>.........] - ETA: 0s - reward: -46.8900Track generation: 1185..1489 -> 304-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 927..1170 -> 243-tiles track
 96/100 [===========

In [14]:
 # After training is done, we save the final weights one more time.
dqn.save_weights(weights_filename, overwrite=True)

In [15]:
# Finally, evaluate our algorithm for 10 episodes.
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Track generation: 1150..1442 -> 292-tiles track
Episode 1: reward: -93.727, steps: 6
Track generation: 1251..1568 -> 317-tiles track
Episode 2: reward: -94.271, steps: 6
Track generation: 1211..1518 -> 307-tiles track
Episode 3: reward: -94.064, steps: 6
Track generation: 1115..1398 -> 283-tiles track
Episode 4: reward: -93.508, steps: 6
Track generation: 1128..1414 -> 286-tiles track
Episode 5: reward: -93.582, steps: 6
Track generation: 1132..1419 -> 287-tiles track
Episode 6: reward: -93.607, steps: 6
Track generation: 1276..1599 -> 323-tiles track
Episode 7: reward: -94.389, steps: 6
Track generation: 1138..1426 -> 288-tiles track
Episode 8: reward: -93.631, steps: 6
Track generation: 1019..1280 -> 261-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1111..1393 -> 282-tiles track
Episode 9: reward: -93.483, steps: 6
Track generation: 985..1240 -> 255-tiles track
Episode 10: reward: -92.726, ste